# Importing Libraries

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import time
from tqdm import tqdm
import matplotlib.pyplot as plt

# Transforming Data

In [2]:
# Set up GPU device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
# Read file
with open("../data/hamlet.txt") as text_file:
  text = text_file.read()

In [4]:
# Create index to character dictionary and vice versa
char_to_idx = {}
idx_to_char = {}

idx = 0
for char in text:
  if char not in char_to_idx:
    char_to_idx[char] = idx
    idx_to_char[idx] = char
    idx += 1

In [5]:
# Change text to character index list
text_idx = [char_to_idx[c] for c in text]
text_idx = torch.LongTensor(text_idx).unsqueeze(dim=1)

# Move data to GPU
text_idx = text_idx.to(device)

# Creating Model

In [6]:
# Define RNN model
class RecurrentNeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size, hidden_size, num_layers):
        super(RecurrentNeuralNetwork, self).__init__()
        self.embedder = nn.Embedding(input_size, input_size)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers)
        self.decoder = nn.Linear(hidden_size, output_size)
    
    def forward(self, input_seq, hidden_state):
        input_seq = self.embedder(input_seq)
        output, hidden_state = self.lstm(input_seq, hidden_state)
        output = self.decoder(output)
        return output, (hidden_state[0].detach(), hidden_state[1].detach())

In [7]:
# Create instance of model on GPU
model = RecurrentNeuralNetwork(input_size=len(char_to_idx), output_size=len(char_to_idx), hidden_size=512, num_layers=3)
model = model.to(device)

# Training Model

In [8]:
# Choose loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [9]:
# Define number of epochs and length of sequence used in RNN
epochs = 25
len_sequence = 100

# Mark start time for training
start_time = time.time()

# Training loop
for epoch in range(epochs):
  print(f"--------------------\nEPOCH: {epoch+1}\n--------------------")

  # Initialize running loss and hidden state
  running_loss = 0
  hidden_state = None

  start_points = range(torch.randint(high=100, size=(1,)).item(), len(text) - len_sequence, len_sequence)

  for start in tqdm(start_points):
    # Define predictor and response
    predictor = text_idx[start:start+len_sequence]
    response = text_idx[start+1:start+len_sequence+1]

    # Predict and compute loss
    prediction, hidden_state = model(predictor, hidden_state)
    loss = loss_fn(torch.squeeze(prediction), torch.squeeze(response))

    # Add to running loss
    running_loss += loss.item()
    
    # Backpropagate to learn weights
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  print(f"Avg. Training Loss: {running_loss/len(start_points):>7f}\n")

  print("Sample Text:")
  print("-----------")
  
  # Initialize hidden state and input text
  hidden_state = None
  random_index = torch.randint(high=len(text)-1, size=(1,)).item()
  input = text_idx[random_index:random_index+1]

  # Sample text loop
  for i in range(200):
      # Compute model output
      logits, hidden_state = model(input, hidden_state)

      # Take softmax to compute probability distribution for next letter and sample
      probabilities = nn.functional.softmax(torch.squeeze(logits), dim=0)
      distribution = torch.distributions.Categorical(probabilities)
      index = distribution.sample()
      
      # Print predicted character
      print(idx_to_char[index.item()], end='')
      
      # Set next input to model
      input[0][0] = index.item()
      
  print("\n-----------\n")
  print(f"Time: {time.time()-start_time:>2f} seconds\n")

--------------------
EPOCH: 1
--------------------


100%|██████████| 1764/1764 [00:29<00:00, 59.02it/s]


Avg. Training Loss: 2.086561

Sample Text:
-----------
adys
(And well th' Kind, th' els proclands
With be a woulls too past us to some.
Then brove all solzows pate now eDculceen and loage
Be shreest th est ut.

HORATIO
A a rimife more imisciemence,
Worm i
-----------

Time: 30.206736 seconds

--------------------
EPOCH: 2
--------------------


100%|██████████| 1764/1764 [00:28<00:00, 60.96it/s]


Avg. Training Loss: 1.577879

Sample Text:
-----------
ts it very
And upon make a stand to more e'es,
treasuly meantly in the dought man's
unlight thy chard by Dewing,
Which fear this kills on commandy. Which halred?
What I fat. Come, in a sorrice and
I d
-----------

Time: 59.459765 seconds

--------------------
EPOCH: 3
--------------------


...

Avg. Training Loss: 0.445431

Sample Text:
-----------
r forced cause,
And, if thou prate of mountains, 'tis
Hamlet. Then she is mad, I pray you live.
If not, you must well.	[All thou 'rst soul.
The praces do, that, I hope, will sir that you have of
enven
-----------

Time: 680.018812 seconds

--------------------
EPOCH: 24
--------------------


100%|██████████| 1765/1765 [00:29<00:00, 60.33it/s]


Avg. Training Loss: 0.454063

Sample Text:
-----------
ow,
An end on his answer of the poison of day,
Have a starr's figures, Ol them step its world be
now; I die those light now himself
his visitaty in his plemy troble to this
Becomes, again, in pander. 
-----------

Time: 709.561344 seconds

--------------------
EPOCH: 25
--------------------


100%|██████████| 1764/1764 [00:29<00:00, 60.23it/s]


Avg. Training Loss: 0.441207

Sample Text:
-----------
;
And that in the boisticious foul rule,
And our solemn diation 't; the King died,
And in this o'erstand's trophet.

OSRIC  She should be all this speak.

HAMLET
Hic excellent conjolver my true and st
-----------

Time: 739.142098 seconds



In [10]:
# Save model parameters
print("Saving model...")
torch.save(model, "../trained_models/hamlet_rnn")

print("Done!")

Saving model...
Done!
